In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')
#%cd ..

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning
# from utils.generate_features import candleCreateNP
# from utils.generate_features import candleCreateNP_vect
from utils.generate_features import candleCreateNP_vect_v2
#from .utils.data_extraction import load_data
#from .utils.data_cleaning import HFDataCleaning

### Load data etc

In [4]:
# print(os.listdir())
path = 'a:/taqhdf5'  #'a:/taqhdf5'
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,32)]).astype(int)
dates = np.array(['20200401','20200402']).astype(int)

# Provide a list of tickers of interest
tickers = ['GOOG']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'trades' # 'trades', 'quotes' or 'both'

# run load_data()
if dataNeeded == 'trades':
    tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'quotes':
    quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'both':
    tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

['.ipynb_checkpoints', 'preprocessing.ipynb', 'testing_and_comparing_candlesticks.ipynb']
8515 ['taq_19930315.h5', 'taq_19930104.h5', 'taq_19930317.h5', 'taq_19930105.h5', 'taq_19930316.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200401
Date, Max: 20200402

##### Data Extraction begins #####

trade data is being extracted..

### Trade Data ###

The raw H5 trade file contains:  ['TradeIndex', 'Trades'] 

Ticker Information:  (b'GOOG            ', 26900500, 71427) 

Sneak peak of the data

            utcsec ex  cond  volume    price TradeStopStockIndicator corr  \
0  14400048517953  P  @ TI      67  1139.44                           

In [5]:
cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])

In [6]:
step = 1#/10 # in minutes

aggregateMinute = np.arange(0,60,step)
aggregateHour = np.arange(9,16,1)
aggregateDate = np.arange(len(cleanedData.Date.unique()))

remove = 30//step

# candle = np.zeros(((len(aggregateDate)*len(aggregateMinute)*len(aggregateHour)),4))
candleNP = np.zeros((((len(aggregateDate)*len(aggregateMinute)*len(aggregateHour))-int(remove*len(aggregateDate))),4))

In [7]:
numpiedData = cleanedData[['Date','Hour','Minute']].to_numpy()
numpiedData = numpiedData.T
numpiedPrice = cleanedData['price'].to_numpy()

### Original

In [23]:
def candleCreateNP():
    ii = 0
    for l in cleanedData.Date.unique():
        for i in aggregateHour:
            for j in aggregateMinute:
                if (i == 9) & (j <30):
                    continue
                
                p1 = numpiedPrice[((numpiedData[0]==l)&\
                                     (numpiedData[1]==i)&\
                                     (numpiedData[2]>=j))&((numpiedData[0]==l)&\
                                                           (numpiedData[1]==i)&\
                                                           (numpiedData[2]<j+step))]
                if len(p1) > 0:
                    candleNP[ii] = np.array([p1[0],p1.max(),p1.min(),p1[-1]])
                else:
                    # if no new prices in the interval considered, use the previous pne
                    candleNP[ii] = candleNP[ii-1]
                ii += 1
                
    return candleNP



In [9]:
# Aggregate data in candle sticks
candleNP = candleCreateNP()

candleNP.shape

NameError: name 'cleanedData' is not defined

In [309]:
# set up as function
def candleCreateNP_vect(verbose=True):
        
    cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
    if verbose:
        print(f"min and max of new hour_min_col: \
              {cleanedData['hour_min_col'].min()}, {cleanedData['hour_min_col'].max()}")
              
    # setup time_bins to group each timestamp
    #print(step)
    delta = step/60
    #print(delta)
    time_bins = np.arange(9.5-delta, 16, delta)
    #print(len(time_bins))
              
    # put each timestamp into a bucket according to time_bins defined by the step variable
    cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False)
    
    # group by date and time_group, extract price, take it first, max, min, last (open, high, low, close)
    OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])              
    
    # return as numpy if preferred
    return OHLC#.values

In [310]:
test_candles=candleCreateNP_vect()

min and max of new hour_min_col:               9.5, 15.983333333333333


In [311]:
test_candles

price                           
                        first      max      min     last
Date     time_group                                     
20200401 0           1122.260  1127.22  1119.51  1126.12
         10          1127.570  1128.00  1120.00  1126.92
         20          1127.190  1127.40  1124.02  1125.44
         30          1125.010  1125.19  1122.97  1123.00
         40          1123.670  1123.68  1120.01  1121.21
...                       ...      ...      ...      ...
20200402 3850        1119.880  1122.88  1117.00  1117.24
         3860        1116.450  1122.96  1112.18  1121.41
         3870        1121.430  1122.86  1119.76  1121.95
         3880        1122.580  1123.60  1121.18  1122.86
         3890        1122.725  1124.00  1119.64  1121.74

[780 rows x 4 columns]

In [337]:
# set up as function
def candleCreateNP_vect_v2(verbose=True,createHoles=False,holes=10,fillHoles=True):
        
    cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
    if verbose:
        print(f"min and max of new hour_min_col: \
              {cleanedData['hour_min_col'].min()}, {cleanedData['hour_min_col'].max()}")
              
    # setup time_bins to group each timestamp
    #print(step)
    delta = step/60
    time_bins = np.arange(9.5-delta, 16, delta)
              
    # put each timestamp into a bucket according to time_bins defined by the step variable
    cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False)
    
    # group by date and time_group, extract price, take it first, max, min, last (open, high, low, close)
    OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])    
    #print(OHLC.index)
    # Create some holes in the candle-table, so we can verify that the method works.
    if createHoles:
        #np.random.seed(2021)
        for dt in OHLC.index.get_level_values(0).unique():
            dropedElements = []
#             print(dt,'\n')
            for inx in np.arange(holes):
                #print(len(time_bins))
                rm = np.random.randint(0,len(time_bins)-2,1,)
                dropedElements.append(rm[0])
                OHLC = OHLC.drop((dt,rm[0]))
#             print('Elements dropped: ',dropedElements)
    #print(len(OHLC.index.get_level_values(1)))
    #print((len(time_bins)-2))
    #Let check if we are missing any values
    dayz = len(OHLC.index.get_level_values(0).unique())
    if len(OHLC.index.get_level_values(1))!=((len(time_bins)-2)*dayz):
        #print('Filling')
        ##### Creating our temporary table, with all the indices that is surposed to be in the actual candle-table.
        ## Creating the multiIndex-index
        mtInd = pd.MultiIndex.from_product([OHLC.index.get_level_values(0).unique(),np.arange(len(time_bins)-2)],
                                   names=['Date','time_group'])

        ## Creating the multiIndex-columns
        mtCol = pd.MultiIndex.from_product([['price'],['first','high','low','last']])

        ## Creating the table itself
        tempDf = pd.DataFrame(np.nan
                              ,columns=mtCol
                              ,index=mtInd)
        #print(tempDf.shape)
        # Filling the non-empty elements of OHLC into the temp-table
        tempDf.loc[OHLC.index]=OHLC.copy(deep=True)

        # To see that the filling mechanism works:
        if fillHoles:

            # Storing the indices to be filled
            toBeFilled = tempDf[tempDf.price['first'].isna()].index

            # Fill out the empty ones!
            tempDf.loc[toBeFilled] = pd.DataFrame({('price','first'):tempDf.price['last'].fillna(method='ffill').loc[toBeFilled],
                                                  ('price','high'):tempDf.price['last'].fillna(method='ffill').loc[toBeFilled],
                                                  ('price','low'):tempDf.price['last'].fillna(method='ffill').loc[toBeFilled],
                                                  ('price','last'):tempDf.price['last'].fillna(method='ffill').loc[toBeFilled]})

        # Return the complete data
        return tempDf
    
    else:
        
        # return as numpy if preferred
        return OHLC#.values

In [16]:
cleanedData

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,TradeReportingFacility,ParticipantTime,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker
0,34200012706854,Q,@ TI,83,1122.26,,00,36579,447,N,,34200012683939,99,0,20200401,2020-04-01 09:30:00.012706,854,9,30,GOOG
1,34200012711048,Q,@ TI,17,1122.26,,00,36580,448,N,,34200012683939,99,0,20200401,2020-04-01 09:30:00.012711,048,9,30,GOOG
2,34200012771200,Q,@ TI,83,1122.26,,00,36583,449,N,,34200012747399,99,0,20200401,2020-04-01 09:30:00.012771,200,9,30,GOOG
3,34200012827903,Q,@ TI,83,1122.26,,00,36584,450,N,,34200012803083,99,0,20200401,2020-04-01 09:30:00.012827,903,9,30,GOOG
4,34200013120215,Q,@ TI,20,1122.08,,00,36589,451,N,,34200013098975,99,0,20200401,2020-04-01 09:30:00.013120,215,9,30,GOOG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42452,57599902907070,Q,@F,200,1120.83,,00,4757671,19870,N,,57599902886754,99,1,20200402,2020-04-02 15:59:59.902907,070,15,59,GOOG
42453,57599942603285,Q,@,100,1121.70,,00,4757773,19871,N,,57599942583595,99,0,20200402,2020-04-02 15:59:59.942603,285,15,59,GOOG
42454,57599942606550,Q,@,665,1121.74,,00,4757774,19872,N,,57599942583595,99,0,20200402,2020-04-02 15:59:59.942606,550,15,59,GOOG
42455,57599999349704,Q,@F,111,1121.74,,00,4757871,19873,N,,57599999331850,99,1,20200402,2020-04-02 15:59:59.999349,704,15,59,GOOG


In [9]:
candles = candleCreateNP_vect_v2(cleanedData
                                 ,0.1)
candles

price                           
                       first     high      low     last
Date     time_group                                    
20200401 0           1122.26  1127.22  1119.51  1126.12
         1           1126.12  1126.12  1126.12  1126.12
         2           1126.12  1126.12  1126.12  1126.12
         3           1126.12  1126.12  1126.12  1126.12
         4           1126.12  1126.12  1126.12  1126.12
...                      ...      ...      ...      ...
20200402 3894        1121.74  1121.74  1121.74  1121.74
         3895        1121.74  1121.74  1121.74  1121.74
         3896        1121.74  1121.74  1121.74  1121.74
         3897        1121.74  1121.74  1121.74  1121.74
         3898        1121.74  1121.74  1121.74  1121.74

[7798 rows x 4 columns]